In [1]:
from typing import Any, List, Union
import logging
import re
from pprint import pprint
from jaf import path
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)



In [2]:
data1 = {
    "users": [
        {
            "name": "Alice",
            "roles": ["admin", "user"],
            "profile": {
                "age": 30,
                "emails": ["alice@example.com", "alice.work@example.com"]
            }
        },
        {
            "name": "Bob",
            "roles": ["user"],
            "profile": {
                "age": 25,
                "emails": ["bob@example.com"]
            }
        },
        {
            "name": "Charlie",
            "roles": ["guest"],
            "profile": {
                "age": 35
            }
        }
    ],
    "settings": {
        "theme": "dark",
        "notifications": True
    }
}


data2 = {
    "users2": [
        {
            "name": "Alice2",
            "roles": ["user"],
            "profile": {
                "age": 30,
                "emails": ["alice2@example.com", "alice.work2@example.com"]
            }
        },
        {
            "name": "Bob2",
            "roles": ["admin", "user"],
            "profile": {
                "age": 35,
                "emails": ["bob2@example.com"]
            }
        },
        {
            "name": "Charlie2",
            "roles": ["guest"],
            "profile": {
                "age": 45
            }
        },
        {
            "name": "David",
            "roles": ["user"],
            "profile": {
                "age": 40
            }
        }
    ],
    "settings": {
        "theme": "dark",
        "notifications": True
    }
}

data = [data1, data2]

In [3]:
data

[{'users': [{'name': 'Alice',
    'roles': ['admin', 'user'],
    'profile': {'age': 30,
     'emails': ['alice@example.com', 'alice.work@example.com']}},
   {'name': 'Bob',
    'roles': ['user'],
    'profile': {'age': 25, 'emails': ['bob@example.com']}},
   {'name': 'Charlie', 'roles': ['guest'], 'profile': {'age': 35}}],
  'settings': {'theme': 'dark', 'notifications': True}},
 {'users2': [{'name': 'Alice2',
    'roles': ['user'],
    'profile': {'age': 30,
     'emails': ['alice2@example.com', 'alice.work2@example.com']}},
   {'name': 'Bob2',
    'roles': ['admin', 'user'],
    'profile': {'age': 35, 'emails': ['bob2@example.com']}},
   {'name': 'Charlie2', 'roles': ['guest'], 'profile': {'age': 45}},
   {'name': 'David', 'roles': ['user'], 'profile': {'age': 40}}],
  'settings': {'theme': 'dark', 'notifications': True}}]

In [4]:
names = path.path_values("*.users.*.name", data)
pprint(names)
print('-'*80)
names_ast = path.path_values_ast(data, ["*", "users", "*", "name"])
pprint(names_ast)

[PathValue(value=Alice, path=['$', '[0]', 'users', '[0]', 'name']),
 PathValue(value=Bob, path=['$', '[0]', 'users', '[1]', 'name']),
 PathValue(value=Charlie, path=['$', '[0]', 'users', '[2]', 'name'])]
--------------------------------------------------------------------------------
[PathValue(value=Alice, path=['$', '[0]', 'users', '[0]', 'name']),
 PathValue(value=Bob, path=['$', '[0]', 'users', '[1]', 'name']),
 PathValue(value=Charlie, path=['$', '[0]', 'users', '[2]', 'name'])]


In [ ]:
emails = path.path_values("**.emails", data)
pprint(emails)

In [6]:
data3 = {
   'name': 'Alice',
   'roles': ['admin', 'user'],
   'test': {
      'test2': {
         'test3': {
            'emails': ['alicedeep@example.com', ['bobdeep@test.com', 'joedeep@test.com']]
         }
      }
   },
   'profile': {
      'age': 30,
      'emails': ['alice@example.com', 'alice.work@example.com']
   }
}

In [7]:
data3

{'name': 'Alice',
 'roles': ['admin', 'user'],
 'test': {'test2': {'test3': {'emails': ['alicedeep@example.com',
     ['bobdeep@test.com', 'joedeep@test.com']]}}},
 'profile': {'age': 30,
  'emails': ['alice@example.com', 'alice.work@example.com']}}

In [8]:
emails2 = path.path_values("profile.emails", data3)
pprint(emails2)

['alice@example.com', 'alice.work@example.com']


In [9]:
emails3 = path.path_values("**.emails", data3)
pprint(emails3)

['alicedeep@example.com',
 ['bobdeep@test.com', 'joedeep@test.com'],
 'alice@example.com',
 'alice.work@example.com']


In [10]:
emails4 = path.path_values("profile.emails.[0]", data3)
pprint(emails4)

[PathValue(value=alice@example.com, path=['$', 'profile', 'emails', '[0]'])]
